In [1]:
import numpy as np

def spa_decoder(var_nodes, check_nodes, llr_channel, max_iter=50):
    """
    Log-Domain Sum-Product Algorithm (SPA) Decoder

    Args:
        var_nodes: List of lists (from Part 1)
        check_nodes: List of lists (from Part 1)
        llr_channel: Received LLRs from channel (numpy array of shape (N,))
        max_iter: Maximum number of iterations

    Returns:
        estimated_bits: Decoded bits (0 or 1)
        success: Boolean, True if syndrome check passed
    """
    n = len(var_nodes)
    m = len(check_nodes)

    # --- 1. 初始化消息结构 ---
    # 我们使用字典或稀疏矩阵来存储消息。
    # key: (row_idx, col_idx), value: message
    # L_vc: Variable-to-Check messages (Phi)
    # L_cv: Check-to-Variable messages (Psi)
    L_vc = {}
    L_cv = {}

    # 初始状态：变量发给校验的消息 = 信道 LLR
    for col in range(n):
        for row in var_nodes[col]:
            L_vc[(row, col)] = llr_channel[col]

    # --- 2. 迭代循环 ---
    for iteration in range(max_iter):

        # === Step A: Check Node Update (Psi) ===
        # 对应 Slide 30
        for row in range(m):
            # 获取该校验节点连接的所有变量节点
            connected_cols = check_nodes[row]

            # 计算 tanh 乘积
            # 优化：可以先算出所有 tanh 的乘积，除以当前的 tanh 得到 "exclude current" 的乘积
            tanh_vals = [np.tanh(L_vc[(row, col)] / 2.0) for col in connected_cols]
            total_prod = np.prod(tanh_vals)

            for i, col in enumerate(connected_cols):
                # 排除自己 (Leave-one-out)
                # 注意：如果 tanh_vals[i] 是 0，除法会报错，这里需要处理数值稳定性
                # 简单做法是重新乘一遍除去自己的
                prod_exclude_me = 1.0
                for j, other_col in enumerate(connected_cols):
                    if i != j:
                        prod_exclude_me *= tanh_vals[j]

                # 防止数值越界 (-1, 1)
                prod_exclude_me = np.clip(prod_exclude_me, -0.999999, 0.999999)

                # 计算 L_cv (Psi)
                L_cv[(row, col)] = 2.0 * np.arctanh(prod_exclude_me)

        # === Step B: Decision & Syndrome Check ===
        # 对应 Slide 31-32
        # 计算后验 LLR (Posterior LLR)
        L_posterior = np.zeros(n)
        estimated_bits = np.zeros(n, dtype=int)

        for col in range(n):
            # 初始信道信息
            sum_val = llr_channel[col]
            # 加上所有校验节点传来的信息
            for row in var_nodes[col]:
                sum_val += L_cv[(row, col)]

            L_posterior[col] = sum_val

            # 硬判决 (Hard Decision)
            # LLR < 0 意味着 1 的概率大 (对于 BPSK +1/-1 映射，通常 LLR<0 对应 bit 1)
            # 注意：这取决于你的 LLR 定义。Slide 29 说 "if Gamma >= 0 then x=0 else x=1"
            if L_posterior[col] < 0:
                estimated_bits[col] = 1
            else:
                estimated_bits[col] = 0

        # 伴随式检查 (Syndrome Check): z = x * H^T
        # 只要检查每个校验方程是否满足 (sum(bits) % 2 == 0)
        syndrome_valid = True
        for row in range(m):
            parity = 0
            for col in check_nodes[row]:
                parity += estimated_bits[col]
            if parity % 2 != 0:
                syndrome_valid = False
                break

        if syndrome_valid:
            return estimated_bits, True

        # === Step C: Variable Node Update (Phi) ===
        # 对应 Slide 32
        for col in range(n):
            for row in var_nodes[col]:
                # 简单的做法：总和 - 来自该行消息
                # L_vc = L_total - L_cv_from_this_row
                # 注意：这里 L_total 用的是当前这轮算出来的 L_posterior
                # 这里的 L_posterior 包含了 Channel + 所有 Check
                # 所以减去当前的 Check 就是 "Extrinsic" 信息
                L_vc[(row, col)] = L_posterior[col] - L_cv[(row, col)]

    # 如果跑完 max_iter 还没对
    return estimated_bits, False

In [2]:
# --- 假设你已经定义了 spa_decoder 函数 ---
# from your_code import spa_decoder

def test_spa_decoder():
    print("=== 开始执行单元测试 (Slide 33-35) ===")

    # 1. 手动构建 Slide 33 的 H 矩阵 (4行, 6列)
    # 这里的索引要非常小心，完全对应矩阵里的 1 的位置

    # var_nodes (列 -> 行)
    # Col 0: 连到 Row 0, 2
    # Col 1: 连到 Row 0, 1
    # Col 2: 连到 Row 1, 3
    # Col 3: 连到 Row 0, 3
    # Col 4: 连到 Row 1, 2
    # Col 5: 连到 Row 2, 3
    var_nodes = [
        [0, 2], [0, 1], [1, 3], [0, 3], [1, 2], [2, 3]
    ]

    # check_nodes (行 -> 列)
    # Row 0: 连到 Col 0, 1, 3
    # Row 1: 连到 Col 1, 2, 4
    # Row 2: 连到 Col 0, 4, 5
    # Row 3: 连到 Col 2, 3, 5
    check_nodes = [
        [0, 1, 3], [1, 2, 4], [0, 4, 5], [2, 3, 5]
    ]

    # 2. 输入信道 LLR (Slide 33)
    gamma = np.array([-0.5, 2.5, -4.0, 5.0, -3.5, 2.5])

    print(f"输入 LLR: {gamma}")

    # 3. 运行译码器
    # 注意：为了能看到中间过程，你可能需要临时修改 spa_decoder
    # 让它在每一轮循环结束时 print(L_posterior)
    # 或者我们只跑 1 轮，再跑 2 轮，再跑 3 轮来查看结果

    # --- 测试第 1 轮 ---
    print("\n--- Testing Iteration 1 ---")
    # 这一步需要你的 decoder 返回后验 LLR (L_posterior) 才能对比
    # 如果你的 decoder 只返回比特，建议加个 debug 模式返回 LLR
    # 这里假设我们手动去核对打印出来的值
    decoded_bits, success = spa_decoder(var_nodes, check_nodes, gamma, max_iter=1)

    expected_iter1 = np.array([-0.2676, 5.0334, -3.7676, 2.2783, -6.2217, -0.7173])
    print(f"Expected LLRs (Slide 34): {expected_iter1}")
    print("请检查你的 decoder 输出是否接近上述值")

    # --- 测试第 3 轮 (最终结果) ---
    print("\n--- Testing Full Decoding (3 Iterations) ---")
    decoded_bits, success = spa_decoder(var_nodes, check_nodes, gamma, max_iter=3)

    expected_bits = np.array([0, 0, 1, 0, 1, 1]) # 对应 LLR > 0 为 0, < 0 为 1
    # 注意 Slide 33 的发送码字 x 是 [0, 0, 1, 0, 1, 1]

    print(f"你的译码结果: {decoded_bits}")
    print(f"期望译码结果: {expected_bits}")

    if np.array_equal(decoded_bits, expected_bits):
        print("✅ 最终比特校验通过！")
    else:
        print("❌ 最终比特校验失败！")

    if success:
        print("✅ 伴随式检测成功 (Syndrome Check Passed)")
    else:
        print("❌ 伴随式检测失败")

# 运行测试
test_spa_decoder()

=== 开始执行单元测试 (Slide 33-35) ===
输入 LLR: [-0.5  2.5 -4.   5.  -3.5  2.5]

--- Testing Iteration 1 ---
Expected LLRs (Slide 34): [-0.2676  5.0334 -3.7676  2.2783 -6.2217 -0.7173]
请检查你的 decoder 输出是否接近上述值

--- Testing Full Decoding (3 Iterations) ---
你的译码结果: [0 0 1 0 1 1]
期望译码结果: [0 0 1 0 1 1]
✅ 最终比特校验通过！
✅ 伴随式检测成功 (Syndrome Check Passed)
